# Extract Transform Load (ETL)
----

## Extract
**Mengambil data berasal dari (https://www.themoviedb.org/) menggunakan API dengan format JSON.Terdapat 3 jenis dataset yang saya ambil, yaitu:**

1.Dataset FILM dengan genre Science Fiction terbaik pada tahun 2018.

https://georgealexandersuwito.github.io/UTS_DataEngineering/df_movie_id_ScienceFiction_2018.json


2.Dataset FILM dengan genre drama terbaik pada tahun 2019.

https://georgealexandersuwito.github.io/UTS_DataEngineering/df_movie_id_drama_2019.json

3.Dataset FILM dengan genre action terbaik pada tahun 2020.

https://georgealexandersuwito.github.io/UTS_DataEngineering/df_movie_id_action_2020.json

### Import Library 

In [12]:
import pandas as pd
import json
import requests
import numpy as np
import ast
import sqlite3 as sq3

In [217]:
from datetime import datetime
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt",'a') as f:
        f.write(timestamp+','+message+'\n')
        

In [215]:
log("Memulai Proses ETL")

In [171]:
log("Extract Phase Dimulai")

### Mempersiapkan API Key dan Movie API

In [172]:
api_key  = "YOUR_API"
movie_api = "https://api.themoviedb.org/3/discover/movie?"
basic_url = 'https://api.themoviedb.org/3/movie/{}?{}' 

In [173]:
log("Membuat fungsi find_id dan find_movie untuk Retrieve data API ")

In [174]:
#fungsi untuk mencari id pada movie dari beberapa page API dengan query
def find_id(page1,page2,q):
    simpan=[]
    for i in range(page1,page2):
        query=f"{q}&page={i}"
        url = movie_api+ api_key+query
       # print(url)
        r = requests.get(url)
        if r.status_code != 200:
            continue
        else:
            data = requests.get(url).json()
            simpan.append(data) 
    df=pd.DataFrame(simpan)
    ar = np.array([])
    ar=np.array([np.append(ar,pd.DataFrame(df['results'][i])['id'].values) for i in range(page1-1,page2-1)])
    ar=ar.flatten()
    return ar

In [175]:
#fungsi untuk mencari film dari hasil fungsi find_id
def find_movie(movie_id):
    json_list = []
    for movie in movie_id:
        url = basic_url.format(movie, api_key)
        r = requests.get(url)
        if r.status_code != 200:
            continue
        else:
            data = r.json()
            json_list.append(data) 
    return json_list


#### Retrieve dataset FILM dengan genre Science Fiction terbaik pada tahun 2018.

In [176]:
log("Retrive dataset FILM dengan genre Science Fiction terbaik pada tahun 2018")

In [177]:
movie_id_SC_2018 = find_id(1,20,"&with_genres=878&primary_release_year=2018&sort_by=vote_average.desc")

In [178]:
df_movie_id_SC_2018 = pd.DataFrame(find_movie(movie_id_SC_2018))

In [179]:
df_movie_id_SC_2018.to_json("df_movie_id_ScienceFiction_2018.json", orient = "records")

In [180]:
df_movie_id_SC_2018.to_csv("df_movie_id_SC_2018.csv",index=False)

#### Retrieve dataset FILM dengan genre drama terbaik pada tahun 2019.

In [181]:
log("Retrieve dataset FILM dengan genre drama terbaik pada tahun 2019")

In [182]:
movie_id_drama_2019=find_id(1,20,"&with_genres=18&primary_release_year=2019&sort_by=vote_average.desc")

In [183]:
df_movie_id_drama_2019 = pd.DataFrame(find_movie(movie_id_drama_2019))

In [184]:
df_movie_id_drama_2019.shape

(380, 25)

In [185]:
df_movie_id_drama_2019.to_json("df_movie_id_drama_2019.json", orient = "records")

In [186]:
df_movie_id_drama_2019.to_csv("df_movie_id_drama_2019.csv",index=False)

#### Retrieve dataset FILM dengan genre action terbaik pada tahun 2020.

In [250]:
log("Retrieve dataset FILM dengan genre action terbaik pada tahun 2020")

In [251]:
movie_id_action_2020=find_id(1,20,"&with_genres=28&primary_release_year=2020&sort_by=vote_average.desc")

In [252]:
df_movie_id_action_2020 = pd.DataFrame(find_movie(movie_id_action_2020))

In [253]:
df_movie_id_action_2020.shape

(380, 25)

In [254]:
df_movie_id_action_2020.to_json("df_movie_id_action_2020.json", orient = "records")

In [255]:
df_movie_id_action_2020.to_csv("df_movie_id_action_2020.csv",index=False)

In [256]:
log("Extract Phase Selesai")

## Transform
**Membagi dataset menjadi 2 Tabel sebelum melakukan proses Transform**

Tabel pertama: `Dataset FILM dengan genre Science Fiction terbaik pada tahun 2018` digabung dengan
`Dataset FILM dengan genre drama terbaik pada tahun 2019`

Tabel kedua:`Tabel Dataset FILM dengan genre action terbaik pada tahun 2020` 

Task yang akan dikerjakan pada tiap tabel:

1.Filtering dan Cleaning:Mengecek nilai null dan menyeleksi fitur pada tabel

2.Deduplication:Mengecek duplikasi data

3.Format revision:Membenahi format yang sesuai

4.Key restructuring: Membagi menjadi beberapa tabel




#### `1.Filtering dan Cleaning`:Mengecek nilai null dan menyeleksi fitur pada tabel

#### Tabel Pertama:
Menggabungkan df_movie_id_SC_2018 dengan df_movie_id_drama_2019

In [868]:
log("Transform Phase Dimulai")

In [206]:
log("Menggabungkan df_movie_id_SC_2018 dengan df_movie_id_drama_2019 menjadi tabel_1")

In [3]:
df_movie_id_SC_2018=pd.read_csv('df_movie_id_SC_2018.csv')
df_movie_id_drama_2019=pd.read_csv('df_movie_id_drama_2019csv')

In [4]:
tabel_1 = pd.concat([df_movie_id_SC_2018,df_movie_id_drama_2019],ignore_index=True)

In [5]:
#mencari null values(angka)
def null_values(df):
    null_values = df.isnull().sum()
    null_values=null_values[null_values>0].sort_values(ascending=False)
    return null_values

In [6]:
#mencari null values(percent)
def percent_null_values(df):
    null_values = df.isnull().sum()
    null_values=null_values[null_values>0].sort_values(ascending=False)/len(df)*100
    return null_values

In [ ]:
log("Mengecek null values pada tabel_1")

In [7]:
null_values(tabel_1)

belongs_to_collection    710
homepage                 487
tagline                  484
backdrop_path            231
imdb_id                  151
overview                  45
runtime                   12
poster_path                6
dtype: int64

In [8]:
percent_null_values(tabel_1)

belongs_to_collection    93.421053
homepage                 64.078947
tagline                  63.684211
backdrop_path            30.394737
imdb_id                  19.868421
overview                  5.921053
runtime                   1.578947
poster_path               0.789474
dtype: float64

In [216]:
log("Melakukan drop column dan rows pada fitur yang banyak null dan kurang informatif pada tabel_1")

Melakukan Drop Column

column | Alasan
--- | ---------
belongs_to_collection | Karena 91% dari keseluruhan data bernilai null
backdrop_path | fitur tersebut kurang informatif
imdb_id | karena sudah terdapat id film.
homepage| kurang informatif karena hanya berisi website film.

Melakukan Drop Rows

rows | Alasan
--- | ---------
runtime | Karena data bernilai null kurang dari 3%
poster_path|Karena data bernilai null kurang dari 3%




In [9]:
drop_column=percent_null_values(tabel_1).index[:-3]

In [10]:
tabel_1=tabel_1.drop(drop_column,axis=1)

In [11]:
null_values(tabel_1)

overview       45
runtime        12
poster_path     6
dtype: int64

In [12]:
tabel_1=tabel_1.dropna()

In [13]:
null_values(tabel_1)

Series([], dtype: int64)

**Seleksi Fitur pada Tabel 1** 

In [209]:
log("Melakukan seleksi fitur pada tabel_1")

In [14]:
tabel_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 705 entries, 0 to 759
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   adult                 705 non-null    bool   
 1   budget                705 non-null    int64  
 2   genres                705 non-null    object 
 3   id                    705 non-null    int64  
 4   original_language     705 non-null    object 
 5   original_title        705 non-null    object 
 6   overview              705 non-null    object 
 7   popularity            705 non-null    float64
 8   poster_path           705 non-null    object 
 9   production_companies  705 non-null    object 
 10  production_countries  705 non-null    object 
 11  release_date          705 non-null    object 
 12  revenue               705 non-null    int64  
 13  runtime               705 non-null    float64
 14  spoken_languages      705 non-null    object 
 15  status                7

Melakukan Drop Column

column | Alasan
--- | ---------
original_title | Karena sudah ada fitur title
video | karena kurang informatif (hanya berisi nilai False)
adult | karena kurang informatif (hanya berisi nilai False)

In [15]:
tabel_1.adult.value_counts()

False    705
Name: adult, dtype: int64

In [16]:
tabel_1.video.value_counts()

False    705
Name: video, dtype: int64

In [17]:
tabel_1=tabel_1.drop(['original_title','video','adult'],axis=1)

In [18]:
tabel_1.shape

(705, 17)

In [19]:
tabel_1.head(2)

,budget,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count
0,0,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",847068,en,A world-class psychiatrist takes on a new pati...,0.632,/4V55Aux76qLE0vu66ZpTp7AjWtr.jpg,[],[],2018-01-01,0,109.0,[],Released,Milgram and the Fastwalkers,10.0,1
1,100,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",807149,es,"Cuscus's grandfather has the virus, so she is ...",0.600,/h2eDPaHiItFid0S1EwVxwRwK2pu.jpg,"[{'id': 139032, 'logo_path': None, 'name': 'Ou...",[],2018-05-20,0,10.0,"[{'english_name': 'Spanish', 'iso_639_1': 'es'...",Released,Cuscus,10.0,1


#### Tabel Kedua:
df_movie_id_action_2020




In [20]:
tabel_2=pd.read_csv('df_movie_id_action_2020.csv')

In [210]:
log("Mengecek null values pada tabel_2")

In [21]:
null_values(tabel_2)

belongs_to_collection    340
homepage                 262
tagline                  237
imdb_id                   67
backdrop_path             58
overview                  23
poster_path                6
dtype: int64

In [22]:
percent_null_values(tabel_2)

belongs_to_collection    89.473684
homepage                 68.947368
tagline                  62.368421
imdb_id                  17.631579
backdrop_path            15.263158
overview                  6.052632
poster_path               1.578947
dtype: float64

In [211]:
log("Melakukan drop column dan rows pada fitur yang banyak null dan kurang informatif pada tabel_2")

Melakukan Drop Column

column | Alasan
--- | ---------
belongs_to_collection | Karena 87% dari keseluruhan data bernilai null
backdrop_path | karena hanya berisi gambar
tagline| karena 60% dari keseluruhan data bernilai null
imdb_id | karena sudah terdapat id film.
homepage| kurang informatif karena hanya berisi website film.




In [23]:
tabel_2=tabel_2.drop(['belongs_to_collection','backdrop_path','tagline','imdb_id','homepage'],axis=1)

In [24]:
tabel_2.shape

(380, 20)

Melakukan Drop Rows

rows | Alasan
--- | ---------
runtime | Karena data bernilai null kurang dari 3%
poster_path|Karena data bernilai null kurang dari 3%


In [25]:
tabel_2=tabel_2.dropna()

In [26]:
null_values(tabel_2)

Series([], dtype: int64)

**Seleksi Fitur Pada Tabel 2**

In [212]:
log("Melakukan seleksi fitur pada tabel_2")

In [27]:
tabel_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 352 entries, 0 to 379
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   adult                 352 non-null    bool   
 1   budget                352 non-null    int64  
 2   genres                352 non-null    object 
 3   id                    352 non-null    int64  
 4   original_language     352 non-null    object 
 5   original_title        352 non-null    object 
 6   overview              352 non-null    object 
 7   popularity            352 non-null    float64
 8   poster_path           352 non-null    object 
 9   production_companies  352 non-null    object 
 10  production_countries  352 non-null    object 
 11  release_date          352 non-null    object 
 12  revenue               352 non-null    int64  
 13  runtime               352 non-null    int64  
 14  spoken_languages      352 non-null    object 
 15  status                3

Melakukan Drop Column

column | Alasan
--- | ---------
original_title | Karena sudah ada fitur title
video | karena kurang informatif (hanya berisi nilai False)
adult | karena kurang informatif (hanya berisi nilai False)

In [28]:
tabel_2=tabel_2.drop(['original_title','video','adult'],axis=1)

In [29]:
tabel_2.shape

(352, 17)

In [30]:
tabel_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 352 entries, 0 to 379
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                352 non-null    int64  
 1   genres                352 non-null    object 
 2   id                    352 non-null    int64  
 3   original_language     352 non-null    object 
 4   overview              352 non-null    object 
 5   popularity            352 non-null    float64
 6   poster_path           352 non-null    object 
 7   production_companies  352 non-null    object 
 8   production_countries  352 non-null    object 
 9   release_date          352 non-null    object 
 10  revenue               352 non-null    int64  
 11  runtime               352 non-null    int64  
 12  spoken_languages      352 non-null    object 
 13  status                352 non-null    object 
 14  title                 352 non-null    object 
 15  vote_average          3

In [249]:
log("Transform Phase Selesai")

#### `2.Deduplication`:Mengecek duplikasi data

In [248]:
log("Deduplication Phase Dimulai")

**Tabel 1**

In [213]:
log("mengecek adanya duplikasi pada tabel_1")

In [33]:
tabel_1[tabel_1.duplicated(subset='id')]

,budget,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count


**Tabel 2**

In [218]:
log("mengecek adanya duplikasi pada tabel_2")

In [34]:
tabel_2[tabel_2.duplicated(subset='id')]

,budget,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count


In [250]:
log("Fase Deduplication Selesai")

#### `3.Format Revision`: Membenahi format yang sesuai

In [251]:
log("Fase Format Revision Dimulai")

**Tabel 1**

In [37]:
tabel_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 705 entries, 0 to 759
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                705 non-null    int64  
 1   genres                705 non-null    object 
 2   id                    705 non-null    int64  
 3   original_language     705 non-null    object 
 4   overview              705 non-null    object 
 5   popularity            705 non-null    float64
 6   poster_path           705 non-null    object 
 7   production_companies  705 non-null    object 
 8   production_countries  705 non-null    object 
 9   release_date          705 non-null    object 
 10  revenue               705 non-null    int64  
 11  runtime               705 non-null    float64
 12  spoken_languages      705 non-null    object 
 13  status                705 non-null    object 
 14  title                 705 non-null    object 
 15  vote_average          7

In [252]:
log("Melakukan penyesuaian format pada fitur yang bertipe angka di tabel_1")

In [38]:
tabel_1['budget']=pd.to_numeric(tabel_1['budget'],errors='coerce')

In [39]:
tabel_1['budget'].value_counts().sort_values(ascending=False)

0           561
50000         7
1000          5
1000000       5
500           4
           ... 
150           1
6248          1
47374         1
442000        1
55000000      1
Name: budget, Length: 98, dtype: int64

In [40]:
tabel_1['budget']=tabel_1['budget'].replace(0,np.nan)

In [41]:
tabel_1['budget']=tabel_1['budget'].div(1000000)

0          NaN
1       0.0001
2          NaN
3          NaN
4          NaN
        ...   
755        NaN
756        NaN
757    55.0000
758     0.0250
759        NaN
Name: budget, Length: 705, dtype: float64

In [42]:
tabel_1['revenue'].value_counts()

0             641
530243742       1
83672673        1
290061297       1
8100000         1
             ... 
24              1
43070915        1
4973            1
956425          1
1074251311      1
Name: revenue, Length: 65, dtype: int64

In [43]:
tabel_1.revenue = tabel_1.revenue.replace(0, np.nan)
tabel_1.revenue = tabel_1.revenue.div(1000000)

In [44]:
tabel_1=tabel_1.rename(columns={'budget':'budget_musd','revenue':'revenue_musd'})

In [45]:
tabel_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 705 entries, 0 to 759
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget_musd           144 non-null    float64
 1   genres                705 non-null    object 
 2   id                    705 non-null    int64  
 3   original_language     705 non-null    object 
 4   overview              705 non-null    object 
 5   popularity            705 non-null    float64
 6   poster_path           705 non-null    object 
 7   production_companies  705 non-null    object 
 8   production_countries  705 non-null    object 
 9   release_date          705 non-null    object 
 10  revenue_musd          64 non-null     float64
 11  runtime               705 non-null    float64
 12  spoken_languages      705 non-null    object 
 13  status                705 non-null    object 
 14  title                 705 non-null    object 
 15  vote_average          7

In [253]:
log("Melakukan penyesuaian format pada fitur yang bertipe waktu di tabel_1")

In [46]:
tabel_1['release_date']=pd.to_datetime(tabel_1['release_date'])

In [47]:
tabel_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 705 entries, 0 to 759
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   budget_musd           144 non-null    float64       
 1   genres                705 non-null    object        
 2   id                    705 non-null    int64         
 3   original_language     705 non-null    object        
 4   overview              705 non-null    object        
 5   popularity            705 non-null    float64       
 6   poster_path           705 non-null    object        
 7   production_companies  705 non-null    object        
 8   production_countries  705 non-null    object        
 9   release_date          705 non-null    datetime64[ns]
 10  revenue_musd          64 non-null     float64       
 11  runtime               705 non-null    float64       
 12  spoken_languages      705 non-null    object        
 13  status              

In [254]:
log("Melakukan penyesuaian format pada fitur yang bertipe object di tabel_1")

In [48]:
tabel_1[tabel_1['overview'].str.isspace()]

,budget_musd,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue_musd,runtime,spoken_languages,status,title,vote_average,vote_count


In [49]:
tabel_1['genres']

0      [{'id': 878, 'name': 'Science Fiction'}, {'id'...
1      [{'id': 878, 'name': 'Science Fiction'}, {'id'...
2      [{'id': 10402, 'name': 'Music'}, {'id': 878, '...
3               [{'id': 878, 'name': 'Science Fiction'}]
4      [{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...
                             ...                        
755    [{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...
756    [{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...
757    [{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...
758                        [{'id': 18, 'name': 'Drama'}]
759    [{'id': 10749, 'name': 'Romance'}, {'id': 18, ...
Name: genres, Length: 705, dtype: object

In [50]:
tabel_1['genres']=tabel_1['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else np.nan)

In [51]:
tabel_1['genres']=tabel_1['genres'].apply(lambda x: "|".join(i['name'] for i in x))

In [52]:
tabel_1['genres']

0                 Science Fiction|Drama|Mystery
1      Science Fiction|Thriller|Western|Fantasy
2                         Music|Science Fiction
3                               Science Fiction
4                        Horror|Science Fiction
                         ...                   
755                       Drama|Romance|History
756           Drama|History|Romance|Fantasy|War
757                        Crime|Thriller|Drama
758                                       Drama
759                               Romance|Drama
Name: genres, Length: 705, dtype: object

In [53]:
tabel_1['production_companies']=tabel_1['production_companies'].apply(lambda x: ast.literal_eval(x))

In [54]:
tabel_1['production_companies']=tabel_1['production_companies'].apply(lambda x:"|".join(i['name'] for i in x ))

In [55]:
#tabel_1['production_companies'].apply(lambda x: print(i) for i in x if len(i)>0 )

In [56]:
tabel_1['production_companies']

0                                                       
1                                        Ouroboros Films
2          Robert Fox and Jones|Tintoretto Entertainment
3                              Grupo Idex|La Boutique 77
4                                            Triler Cine
                             ...                        
755                                          MD Pictures
756                                                  PSC
757    Warner Bros. Pictures|Joint Effort|Village Roa...
758                                   Hohoww Productions
759                                                  MTA
Name: production_companies, Length: 705, dtype: object

In [57]:
tabel_1['production_countries'] = tabel_1['production_countries'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else np.nan )

In [58]:
tabel_1['production_countries']

0                                                     []
1                                                     []
2      [{'iso_3166_1': 'GB', 'name': 'United Kingdom'...
3                [{'iso_3166_1': 'ES', 'name': 'Spain'}]
4                                                     []
                             ...                        
755          [{'iso_3166_1': 'ID', 'name': 'Indonesia'}]
756              [{'iso_3166_1': 'JP', 'name': 'Japan'}]
757    [{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...
758    [{'iso_3166_1': 'US', 'name': 'United States o...
759             [{'iso_3166_1': 'TR', 'name': 'Turkey'}]
Name: production_countries, Length: 705, dtype: object

In [59]:
tabel_1['production_countries']=tabel_1['production_countries'].apply(lambda x: "|".join(i['name'] for i in x) )

In [60]:
tabel_1['production_countries']

0                                             
1                                             
2      United Kingdom|United States of America
3                                        Spain
4                                             
                        ...                   
755                                  Indonesia
756                                      Japan
757            Canada|United States of America
758                   United States of America
759                                     Turkey
Name: production_countries, Length: 705, dtype: object

In [61]:
tabel_1['spoken_languages']

0                                                     []
1      [{'english_name': 'Spanish', 'iso_639_1': 'es'...
2      [{'english_name': 'English', 'iso_639_1': 'en'...
3      [{'english_name': 'English', 'iso_639_1': 'en'...
4      [{'english_name': 'Spanish', 'iso_639_1': 'es'...
                             ...                        
755    [{'english_name': 'Indonesian', 'iso_639_1': '...
756    [{'english_name': 'Japanese', 'iso_639_1': 'ja...
757    [{'english_name': 'English', 'iso_639_1': 'en'...
758    [{'english_name': 'English', 'iso_639_1': 'en'...
759    [{'english_name': 'Turkish', 'iso_639_1': 'tr'...
Name: spoken_languages, Length: 705, dtype: object

In [62]:
tabel_1['spoken_languages'] = tabel_1['spoken_languages'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else np.nan )

In [63]:
tabel_1['spoken_languages'] = tabel_1['spoken_languages'].apply(lambda x: "|".join(i['name'] for i in x) if isinstance(x,list) else np.nan )

In [64]:
tabel_1['spoken_languages']

0                      
1               Español
2               English
3               English
4               Español
             ...       
755    Bahasa indonesia
756                 日本語
757             English
758             English
759              Türkçe
Name: spoken_languages, Length: 705, dtype: object

In [65]:
col_tabel = ["id", "title", "release_date", "genres", 
       "original_language", "budget_musd", "revenue_musd", "production_companies",
       "production_countries", "vote_count", "vote_average", "popularity", "runtime",
       "overview", "spoken_languages", "poster_path"]

In [66]:
tabel_1=tabel_1.loc[:,col_tabel]

In [67]:
tabel_1.dropna(thresh = 14, inplace = True)

In [68]:
tabel_1['production_companies']=tabel_1['production_companies'].replace('',np.nan)

In [69]:
tabel_1['production_countries']=tabel_1['production_countries'].replace('',np.nan)

In [70]:
tabel_1['spoken_languages']=tabel_1['spoken_languages'].replace('',np.nan)

In [71]:
tabel_1['overview'].loc[tabel_1['overview'].str.isspace()]

Series([], Name: overview, dtype: object)

In [72]:
tabel_1[tabel_1['overview']=='']

,id,title,release_date,genres,original_language,budget_musd,revenue_musd,production_companies,production_countries,vote_count,vote_average,popularity,runtime,overview,spoken_languages,poster_path


In [73]:
tabel_1[tabel_1['overview']==' ']

,id,title,release_date,genres,original_language,budget_musd,revenue_musd,production_companies,production_countries,vote_count,vote_average,popularity,runtime,overview,spoken_languages,poster_path


In [74]:
tabel_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 705 entries, 0 to 759
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    705 non-null    int64         
 1   title                 705 non-null    object        
 2   release_date          705 non-null    datetime64[ns]
 3   genres                705 non-null    object        
 4   original_language     705 non-null    object        
 5   budget_musd           144 non-null    float64       
 6   revenue_musd          64 non-null     float64       
 7   production_companies  485 non-null    object        
 8   production_countries  565 non-null    object        
 9   vote_count            705 non-null    int64         
 10  vote_average          705 non-null    float64       
 11  popularity            705 non-null    float64       
 12  runtime               705 non-null    float64       
 13  overview            

In [75]:
tabel_1.head()

,id,title,release_date,genres,original_language,budget_musd,revenue_musd,production_companies,production_countries,vote_count,vote_average,popularity,runtime,overview,spoken_languages,poster_path
0,847068,Milgram and the Fastwalkers,2018-01-01,Science Fiction|Drama|Mystery,en,NaN,NaN,NaN,NaN,1,10.0,0.632,109.0,A world-class psychiatrist takes on a new pati...,NaN,/4V55Aux76qLE0vu66ZpTp7AjWtr.jpg
1,807149,Cuscus,2018-05-20,Science Fiction|Thriller|Western|Fantasy,es,100.0,NaN,Ouroboros Films,NaN,1,10.0,0.600,10.0,"Cuscus's grandfather has the virus, so she is ...",Español,/h2eDPaHiItFid0S1EwVxwRwK2pu.jpg
2,780614,Lazarus,2018-05-02,Music|Science Fiction,en,NaN,NaN,Robert Fox and Jones|Tintoretto Entertainment,United Kingdom|United States of America,1,10.0,2.183,110.0,Lazarus is a musical with music and lyrics com...,English,/oMNM5ArncJbLJvyqeHnlJZFGmf2.jpg
3,767757,IPDENTICAL: Imagine a world without creativity,2018-05-29,Science Fiction,en,NaN,NaN,Grupo Idex|La Boutique 77,Spain,1,10.0,0.600,9.0,Anna lives in a retro-futuristic dystopia wher...,English,/57x22PQ5Nmr91BOiKWgdxAx5Wjp.jpg
4,762812,Nocturne,2018-11-29,Horror|Science Fiction,es,NaN,NaN,Triler Cine,NaN,1,10.0,0.600,9.0,"When everything seemed to be going normally, a...",Español,/y3smngJh0IyDj6SjUAntFVC4IxA.jpg


In [76]:
tabel_1.shape

(705, 16)

**Tabel 2**

In [255]:
log("Melakukan penyesuaian format pada fitur yang bertipe angka di tabel_2")

In [77]:
tabel_2['revenue']=pd.to_numeric(tabel_2['revenue'])

In [78]:
tabel_2['revenue']=tabel_2['revenue'].div(1000000)

In [79]:
tabel_2['revenue']=tabel_2['revenue'].replace(0,np.nan)

In [80]:
tabel_2['budget']=pd.to_numeric(tabel_2['budget'])

In [81]:
tabel_2['budget']=tabel_2['budget'].div(1000000)

In [82]:
tabel_2['budget']=tabel_2['budget'].replace(0,np.nan)

In [83]:
tabel_2=tabel_2.rename(columns={'budget':'budget_musd','revenue':'revenue_musd'})

In [84]:
col = ['production_companies','production_countries','genres','spoken_languages']

In [256]:
log("Melakukan penyesuaian format pada fitur yang bertipe object di tabel_2")

In [85]:
for i in col:
    tabel_2[i]=tabel_2[i].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else np.nan)
    tabel_2[i]=tabel_2[i].apply(lambda x: "|".join(i['name'] for i in x) if isinstance(x,list) else np.nan)

In [86]:
tabel_2['production_companies']=tabel_2['production_companies'].replace('',np.nan)

In [87]:
tabel_2['production_countries']=tabel_2['production_countries'].replace('',np.nan)

In [88]:
tabel_2['spoken_languages']=tabel_2['spoken_languages'].replace('',np.nan)

In [257]:
log("Melakukan penyesuaian format pada fitur yang bertipe waktu di tabel_2")

In [89]:
tabel_2['release_date']=pd.to_datetime(tabel_2['release_date'])

In [90]:
null_values(tabel_2)

revenue_musd            303
budget_musd             292
production_companies     93
production_countries     45
spoken_languages         37
dtype: int64

In [91]:
tabel_2=tabel_2.loc[:,col_tabel]

In [92]:
tabel_2=tabel_2.dropna(thresh=14)

In [93]:
null_values(tabel_2)

revenue_musd            192
budget_musd             186
production_companies      4
production_countries      3
spoken_languages          1
dtype: int64

In [94]:
tabel_2.shape

(241, 16)

In [95]:
tabel_2.head()

,id,title,release_date,genres,original_language,budget_musd,revenue_musd,production_companies,production_countries,vote_count,vote_average,popularity,runtime,overview,spoken_languages,poster_path
0,863519,The Protectors,2020-04-24,Science Fiction|Action,en,NaN,NaN,Limitbreaker Studios|Skyward Stories,United States of America,1,10.0,1.201,90,After finding out the truth about his alien an...,English,/5qs8CrOWBIARlLhsIdueDpT5xiC.jpg
3,853543,Assassin's Vow,2020-12-01,Action,en,3.5,NaN,Pixium Film Group|Safier Entertainment,United States of America,1,10.0,2.503,94,"Set in the crime underworld of South Florida, ...",English,/c93bJ880nkbaKEP9cush10Nz4fS.jpg
6,824536,Motu Patlu in the Game of Zones,2020-12-19,Animation|Comedy|Mystery|Fantasy|Adventure|Action,hi,NaN,NaN,Voot Kids|Nickelodeon (IN)|Viacom 18 Motion Pi...,India,1,10.0,0.600,87,A board game becomes all too real for buddies ...,हिन्दी|தமிழ்|తెలుగు,/xdoQSrB0kzIRT9CWEbMbX2vHYbB.jpg
15,775241,Good Times Bad Times,2020-09-25,Action|Crime,pt,NaN,NaN,RUGAS,Portugal,2,10.0,0.600,6,"The good times we wished we had lived, might h...",Português,/ba4PQ59z7oJ8d5C26rkH1LLqYJc.jpg
17,771390,Macabre Night,2020-12-03,Horror|Action|Music|Comedy,pt,NaN,NaN,Abóbora Filmes|Mina Audiovisual,Brazil,2,10.0,0.840,20,"At night, during a transfer, the inmates of a ...",Português,/jJMZjnP26Ih8eeC4oP4RgmY2IFF.jpg


In [258]:
log("Fase Format Revision Selesai")

#### `4.Key restructuring`

**Tabel 1**

In [259]:
log("Fase Key restructuring Dimulai")

In [260]:
log("Menggabungkan tabel_1 dengan tabel_2")

In [225]:
files=['df_movie_id_SC_2018.json','df_movie_id_drama_2019.json','df_movie_id_action_2020.json']

def merge_JsonFiles(filename):
    result = list()
    for f1 in filename:
        with open(f1, 'r') as infile:
            result.extend(json.load(infile))

    with open('Tabel_1.json', 'w') as output_file:
        json.dump(result, output_file)

merge_JsonFiles(files)

In [2]:
with open('Tabel_1.json') as f:
    data=json.load(f)

In [3]:
data

[{'adult': False,
  'backdrop_path': None,
  'belongs_to_collection': None,
  'budget': 0,
  'genres': [{'id': 878, 'name': 'Science Fiction'},
   {'id': 18, 'name': 'Drama'},
   {'id': 9648, 'name': 'Mystery'}],
  'homepage': '',
  'id': 847068,
  'imdb_id': None,
  'original_language': 'en',
  'original_title': 'Milgram and the Fastwalkers',
  'overview': 'A world-class psychiatrist takes on a new patient who is experiencing strange dreams, and is reluctantly drawn into the dark world of aliens and UFOs.',
  'popularity': 0.6,
  'poster_path': '/4V55Aux76qLE0vu66ZpTp7AjWtr.jpg',
  'production_companies': [],
  'production_countries': [],
  'release_date': '2018-01-01',
  'revenue': 0,
  'runtime': 109.0,
  'spoken_languages': [],
  'status': 'Released',
  'tagline': '',
  'title': 'Milgram and the Fastwalkers',
  'video': False,
  'vote_average': 10.0,
  'vote_count': 1},
 {'adult': False,
  'backdrop_path': '/vX5oa5xup7DuUXvckQZgUsxFg9G.jpg',
  'belongs_to_collection': None,
  'budg

In [231]:
Tabel_1 = pd.json_normalize(data, sep = "_")

In [232]:
Tabel_1=Tabel_1.sort_values('revenue',ascending=False)

In [261]:
log("Key restructing tabel movies")

In [233]:
movies = Tabel_1[["id", "title", "revenue", "budget", "belongs_to_collection_name", "release_date"]].copy()

In [234]:
movies.head()

,id,title,revenue,budget,belongs_to_collection_name,release_date
37,299536,Avengers: Infinity War,2046239637,300000000,The Avengers Collection,2018-04-25
67,284054,Black Panther,1346739107,200000000,Black Panther Collection,2018-02-13
142,351286,Jurassic World: Fallen Kingdom,1303459585,170000000,Jurassic Park Collection,2018-06-06
597,475557,Joker,1074251311,55000000,NaN,2019-10-02
125,335983,Venom,855013954,116000000,Venom Collection,2018-09-28


In [122]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600 entries, 37 to 599
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   id                          600 non-null    int64 
 1   title                       600 non-null    object
 2   revenue                     600 non-null    int64 
 3   budget                      600 non-null    int64 
 4   belongs_to_collection_name  41 non-null     object
 5   release_date                600 non-null    object
dtypes: int64(3), object(3)
memory usage: 32.8+ KB


In [235]:
movies['release_date']=pd.to_datetime(movies['release_date'])

In [236]:
movies['revenue'] = df.revenue/1000000
movies['budget'] = df.budget/1000000

In [237]:
movies.head()

,id,title,revenue,budget,belongs_to_collection_name,release_date
37,299536,Avengers: Infinity War,2046.239637,300.0,The Avengers Collection,2018-04-25
67,284054,Black Panther,1346.739107,200.0,Black Panther Collection,2018-02-13
142,351286,Jurassic World: Fallen Kingdom,1303.459585,170.0,Jurassic Park Collection,2018-06-06
597,475557,Joker,1074.251311,55.0,NaN,2019-10-02
125,335983,Venom,855.013954,116.0,Venom Collection,2018-09-28


In [262]:
log("Key restructing tabel votes")

In [238]:
votes = df[['id','vote_count','vote_average']].copy()
votes

,id,vote_count,vote_average
0,847068,1,10.0
1,807149,1,10.0
2,780614,1,10.0
3,767757,1,10.0
4,762812,1,10.0
...,...,...,...
595,611369,5,8.2
596,616538,11,8.2
597,475557,18868,8.2
598,730980,3,8.2


In [264]:
log("Key restructing tabel genres")

In [4]:
genres=pd.json_normalize(data,record_path='genres',meta='id',record_prefix='genre_')
genres

,genre_id,genre_name,id
0,878,Science Fiction,847068
1,18,Drama,847068
2,9648,Mystery,847068
3,878,Science Fiction,807149
4,53,Thriller,807149
...,...,...,...
2373,10752,War,493550
2374,35,Comedy,493550
2375,28,Action,663255
2376,12,Adventure,663255


In [265]:
log("Key restructing tabel prod")

In [240]:
prod=pd.json_normalize(data,record_path='production_companies',meta='id',record_prefix='comp_')
prod.head()

,comp_id,comp_logo_path,comp_name,comp_origin_country,id
0,139032,None,Ouroboros Films,,807149
1,146418,None,Robert Fox and Jones,,780614
2,146419,None,Tintoretto Entertainment,,780614
3,148656,None,Grupo Idex,,767757
4,148657,None,La Boutique 77,,767757


In [241]:
prod=prod.drop_duplicates(subset='comp_id')

In [242]:
prod[prod.duplicated()]

,comp_id,comp_logo_path,comp_name,comp_origin_country,id


In [243]:
prod.head()

,comp_id,comp_logo_path,comp_name,comp_origin_country,id
0,139032,None,Ouroboros Films,,807149
1,146418,None,Robert Fox and Jones,,780614
2,146419,None,Tintoretto Entertainment,,780614
3,148656,None,Grupo Idex,,767757
4,148657,None,La Boutique 77,,767757


In [266]:
log("Fase Key restructuring Selesai")

## Load


In [267]:
log("Load Phase Dimulai")

**Tabel 1**

In [268]:
log("Membuat koneksi database")

In [13]:
con = sq3.connect("movies_database.db")

In [270]:
log("Meload tabel ke dalam database")

In [277]:
movies.to_sql("Movies", con, index = False)

In [278]:
votes.to_sql("Votes", con, index = False)

In [279]:
genres.to_sql("Genres", con, index = False)

In [280]:
prod.to_sql("Prod", con, index = False)

In [281]:
con.execute("Select * FROM sqlite_master").fetchall()

[('table',
  'Movies',
  'Movies',
  2,
  'CREATE TABLE "Movies" (\n"id" INTEGER,\n  "title" TEXT,\n  "revenue" REAL,\n  "budget" REAL,\n  "belongs_to_collection_name" TEXT,\n  "release_date" TIMESTAMP\n)'),
 ('table',
  'Votes',
  'Votes',
  17,
  'CREATE TABLE "Votes" (\n"id" INTEGER,\n  "vote_count" INTEGER,\n  "vote_average" REAL\n)'),
 ('table',
  'Genres',
  'Genres',
  21,
  'CREATE TABLE "Genres" (\n"genre_id" INTEGER,\n  "genre_name" TEXT,\n  "id" INTEGER\n)'),
 ('table',
  'Prod',
  'Prod',
  34,
  'CREATE TABLE "Prod" (\n"comp_id" INTEGER,\n  "comp_logo_path" TEXT,\n  "comp_name" TEXT,\n  "comp_origin_country" TEXT,\n  "id" INTEGER\n)')]

In [282]:
con.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name").fetchall()

[('Genres',), ('Movies',), ('Prod',), ('Votes',)]

In [14]:
con.close()

In [271]:
log("Load Phase Selesai")

## Proses Query berada pada file "SQL_Query_195150201111015"

----
----